# OpenAI API

In [42]:
from openai import OpenAI
client = OpenAI(api_key="api-key-here")

audio_file= open("marathi.mp3", "rb")
transcript = client.audio.translations.create(
  model="whisper-1", 
  file=audio_file
)


In [43]:
transcript.text


'Hi, my name is Rishi, I have a headache, I am feeling hot and my back is aching. I am very bored.'

In [44]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an intern in the healthcare domain and have been tasked to take a patient's medical history."},
    {"role": "user", "content": f"The patient has provided the following history so far '{transcript.text}'. Now based on this patient's medical history, can you give me a list of questions i have to ask to the patient to get more information from them, so that doctor has all the information to help the patient. Give me the questions in the following format 'question#: question' Don't say anything other than the above format. don't format the text . give me only 3 follow up questions"},
  ]
)


In [45]:
for q in (response.choices[0].message.content.split("\n")):
    print(q)


question1: When did the headache start?
question2: Have you taken any medication for the headache?
question3: On a scale of 1 to 10, what is the intensity of your headache?


In [46]:
audio_file= open("hindi-followup.mp3", "rb")
transcript_followup = client.audio.translations.create(
  model="whisper-1", 
  file=audio_file
)


In [47]:
transcript_followup.text


"I've been having a headache for the past 2 weeks and it's intensity is 7. I haven't tried a lot of medications and I've been using the same remedies at home. I used to drink hot water. And my whole body is aching and I'm feeling hot."

In [48]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an intern in the healthcare domain and have been tasked to take a patient's medical history."},
    {"role": "user", "content": f"The patient has provided the following history so far '{transcript.text}' and '{transcript_followup.text}'. convert this into a doctor's notes format with sections for only the details you have without a plan of action. don't add a note for this being fictional, that is acceptable. Don't add any text formatting"},
  ]
)


In [49]:
for q in (response.choices[0].message.content.split("\n")):
    print(q)


Patient Name: Rishi

Chief Complaints:
1. Headache: 2 weeks duration, intensity 7/10
2. Feeling hot
3. Backache
4. Boredom

History of Present Illness:
The patient, Rishi, presents with a persistent headache over the past two weeks, with an intensity rating of 7 out of 10. Patient also reports feeling hot and experiencing aching throughout the entire body. Patient mentions that they have been using home remedies, including drinking hot water, to alleviate their symptoms. Additionally, the patient expresses a sense of boredom.

Medication History:
Patient has not tried many medications for their headache and other symptoms.

Additional Notes:
No further information available at this time.


In [50]:

client = boto3.client(service_name='comprehendmedical')


'Patient Name: RishiChief Complaint: Headache, feeling hot, back pain, and generalized body acheHistory of Present Illness:The patient reports experiencing a headache for the past 2 weeks with an intensity level of 7 out of 10. The headache is accompanied by a feeling of heat and aching in the entire body. The patient has not tried many medications, relying primarily on home remedies such as drinking hot water.Past Medical History: Review of Systems: - Neurological: Headache, intensity level 7/10- Musculoskeletal: Back pain- General: Feeling hot, generalized body ache- Psychiatric: BoredomAllergies: No known allergiesMedications: No current medicationsSocial History: No significant social historyFamily History: No significant family medical history providedConclusion: The patient, Rishi, presents with a persistent headache, feeling hot, back pain, and generalized body ache. The headache has been occurring for the past 2 weeks with an intensity level of 7 out of 10. The patient has not 

'bbc'

In [51]:
result = client.detect_entities(Text=response.choices[0].message.content)


In [41]:
for ent in result['Entities']:
    print(ent.get('Category'), ent.get('Text'), ent.get('Type', []), ent.get('Attributes', []))


PROTECTED_HEALTH_INFORMATION Rishi NAME []
MEDICAL_CONDITION Headache DX_NAME []
MEDICAL_CONDITION feeling hot DX_NAME []
ANATOMY back SYSTEM_ORGAN_SITE []
MEDICAL_CONDITION back pain DX_NAME []
MEDICAL_CONDITION generalized body ache DX_NAME []
ANATOMY body SYSTEM_ORGAN_SITE []
MEDICAL_CONDITION headache DX_NAME [{'Type': 'ACUITY', 'Score': 0.7726567387580872, 'RelationshipScore': 0.9999920129776001, 'RelationshipType': 'ACUITY', 'Id': 17, 'BeginOffset': 179, 'EndOffset': 191, 'Text': 'past 2 weeks', 'Category': 'MEDICAL_CONDITION', 'Traits': []}]
MEDICAL_CONDITION past 2 weeks ACUITY []
TEST_TREATMENT_PROCEDURE intensity level TEST_NAME [{'Type': 'TEST_VALUE', 'Score': 0.586040735244751, 'RelationshipScore': 0.9955652356147766, 'RelationshipType': 'TEST_VALUE', 'Id': 42, 'BeginOffset': 219, 'EndOffset': 230, 'Text': '7 out of 10', 'Category': 'TEST_TREATMENT_PROCEDURE', 'Traits': []}]
MEDICAL_CONDITION headache DX_NAME [{'Type': 'ACUITY', 'Score': 0.7726567387580872, 'RelationshipSco